In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


# Gaussian Discriminative Analysis
### Generative Learning Algorithm vs Discriminitive Learning Algorithm
Alogirthm that tries to learn mappings from input space $ \mathcal{x} $ to the output labels $ \mathcal{y} $ ( such as logistic regression, linear regression etc.) are called **Discriminitive Learning Algorithm(DLA)**. In other wards discriminitive learning algorithm tries to learn $ y $ given $ x $. Mathemetically $ P(y | x) $. On the other hand, algorithm that tries to learn $ x $ given $ y $ $ (i. e. p(x | y)) $ is called **Generative Learning Algorithm (GLA)**. Naive Bayes, Gaussian discriminant analysis are the example of GLA. While DLA tries to find a dicision boundry based on the input data, GLA tries to fit a gaussian in each input feature. 
![DLA vs GLA](img/GLA_DLA.png)
You can learn more about the differences [here](https://medium.com/@mlengineer/generative-and-discriminative-models-af5637a66a3#:~:text=In%20General%2C%20A%20Discriminative%20model,actual%20distribution%20of%20each%20class.&text=A%20Generative%20Model%20%E2%80%8Clearns%20the,the%20help%20of%20Bayes%20Theorem.). 

### Multivariate Gaussian Distribution
Gaussian Discriminant Analysis model assumes that $ p(x | y) $ is distributed according to a multivariate normal distribution, which is parameterized by a **mean vector** $ \mu \in \mathbb{R}^n $ and a **covariance matrix** $ \Sigma \in \mathbb{R}^{nxn} $. This can also be written as $ \mathcal{N}(\mu, \Sigma) $ and it's density function can be given by:
$$ 
p(x;\mu,\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}exp\Bigg(-\frac{1}{2}\big(x - \mu\big)^T\Sigma^{-1}\big(x - \mu\big)\Bigg)
$$
The mean vector and covariance matrix will determine the shape of the probability density function. In the following section you can play with these parameter.

In [24]:
@interact(mu1=(-3,3,0.1),  mu2=(-3,3.0,0.1), diagonal_1=(0,3.0,0.1), diagonal_2=(0,3.0,0.1), non_diagonal=(-3,3.0,0.1))
def visualize_multivariate_gaussian(mu1=0.0, mu2=0.0, diagonal_1=1, diagonal_2=1, non_diagonal=.5):
    # This code snippet is taken from here [https://scipython.com/blog/visualizing-the-bivariate-gaussian-distribution/]
    N = 300
    X = np.linspace(-3, 3, N)
    Y = np.linspace(-3, 4, N)
    X, Y = np.meshgrid(X, Y)

    # Mean vector and covariance matrix
    mu = np.array([mu1, mu2])
    Sigma = np.array([[ diagonal_1 , non_diagonal], [non_diagonal,  diagonal_2]])

    # Pack X and Y into a single 3-dimensional array
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X
    pos[:, :, 1] = Y

    def multivariate_gaussian(pos, mu, Sigma):
        """Return the multivariate Gaussian distribution on array pos.

        pos is an array constructed by packing the meshed arrays of variables
        x_1, x_2, x_3, ..., x_k into its _last_ dimension.

        """

        n = mu.shape[0]
        Sigma_det = np.linalg.det(Sigma)
        Sigma_inv = np.linalg.inv(Sigma)
        N = np.sqrt((2*np.pi)**n * Sigma_det)
        # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
        # way across all the input variables.
        fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

        return np.exp(-fac / 2) / N

    # The distribution on the variables X, Y packed into pos.
    Z = multivariate_gaussian(pos, mu, Sigma)

    # Create a surface plot and projected filled contour plot under it.
    fig = plt.figure(figsize=(15,10))
    ax = fig.gca(projection='3d')
    ax.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True,
                    cmap=cm.viridis)

    cset = ax.contourf(X, Y, Z, zdir='z', offset=-0.15, cmap=cm.viridis)

    # Adjust the limits, ticks and view angle
    ax.set_zlim(-0.15,0.2)
    ax.set_zticks(np.linspace(0,0.2,5))
    ax.view_init(27, -21)

    plt.show()
    

interactive(children=(FloatSlider(value=0.0, description='mu1', max=3.0, min=-3.0), FloatSlider(value=0.0, des…

### Gaussian Discriminant Analysis(GDA) model
GDA is perfect for the case where the problem is a classificaiton problem and input variable is continious and fall into a gaussian distribution. For simplicisity, lets assume we are working on a binary classification problem ( $ \mathcal{y} \in \{0, 1\} $). We will apply GDA model which will model $ p(x|y) $ using a multivariate normal distribution. The model is:
$$
y ~ Bernoulli(\phi)
$$